In [1]:
import sagemaker
import os

In [2]:
from sagemaker.tensorflow import TensorFlow

In [3]:
sess = sagemaker.Session()
role = sagemaker.get_execution_role()

In [4]:
data_input_path = sess.upload_data('data/severstal-binary')
print(data_input_path)

s3://sagemaker-us-east-2-475496805360/data


In [5]:
training_input_path = data_input_path + "/train"
validation_input_path = data_input_path + "/validate"

In [6]:
tf_estimator = TensorFlow(
    entry_point='fine_tune_eff_net.py', 
    role=role,
    instance_count=1, 
    instance_type='local',
    framework_version='2.3', 
    py_version='py37',
    script_mode=True,
    hyperparameters={'epochs': 5}
)

In [7]:
tf_estimator.fit({'training': training_input_path, 'validation': validation_input_path})

Creating tmp14rthhzi_algo-1-yjtax_1 ... 
Attaching to tmp14rthhzi_algo-1-yjtax_12mdone
algo-1-yjtax_1  | 2020-11-20 14:25:08,167 sagemaker-training-toolkit INFO     Imported framework sagemaker_tensorflow_container.training
algo-1-yjtax_1  | 2020-11-20 14:25:08,175 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
algo-1-yjtax_1  | 2020-11-20 14:25:09,838 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
algo-1-yjtax_1  | 2020-11-20 14:25:09,861 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
algo-1-yjtax_1  | 2020-11-20 14:25:09,882 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
algo-1-yjtax_1  | 2020-11-20 14:25:09,895 sagemaker-training-toolkit INFO     Invoking user script
algo-1-yjtax_1  | 
algo-1-yjtax_1  | Training Env:
algo-1-yjtax_1  | 
algo-1-yjtax_1  | {
algo-1-yjtax_1  |     "additional_framework_parameters": {},
algo-1-yjtax_1  |     "cha

In [8]:
import efficientnet.keras as efn

Using TensorFlow backend.


In [9]:
import tensorflow as tf
import keras
from keras import backend as K
from keras import models
from keras import layers
from keras.optimizers import SGD
from keras.utils import multi_gpu_model

from tensorflow.keras import applications
from tensorflow.keras.preprocessing import image

def get_model(image_shape):
    inputs = layers.Input(shape=(*image_shape, 3))
    
    Net = efn.EfficientNetB0
    base_efficient_net = Net(weights='imagenet', input_tensor=inputs, include_top=False)

    base_efficient_net.trainable = False

    x = base_efficient_net.output
    x = layers.GlobalAveragePooling2D()(x)

    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    predictions = layers.Dense(2, activation='softmax')(x)

    efficient_net = models.Model(inputs=base_efficient_net.input, outputs=predictions)

    efficient_net.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return efficient_net


In [10]:
get_model((224, 224))